In [ ]:
import pandas as pd
import os
from PIL import Image, ImageFilter
from PIL.Image import Resampling as ImageResampling
import numpy as np

def process_image(image_path, output_folder, category_id, categories, size=(224, 224), sharpness_threshold=100):
        try:
            with Image.open(image_path) as img:
                img = img.resize(size, ImageResampling.LANCZOS)
                rgb_img = img.convert('RGB')
                category_folder = os.path.join(output_folder, categories.get(category_id, 'Unknown'))
                if not os.path.exists(category_folder):
                    os.makedirs(category_folder)
                output_path = os.path.join(category_folder, os.path.basename(image_path))
                rgb_img.save(output_path, 'JPEG')
                print(f"Processed and moved {os.path.basename(output_path)} to {category_folder}")
        except Exception as e:
            print(f"Failed to process image {image_path}: {str(e)}")

def load_categories(category_file):
    category_dict = {}
    with open(category_file, 'r') as file:
        next(file)  # Skip the header line
        for line in file:
            category_id, category_name = line.strip().split('\t')
            category_dict[int(category_id)] = category_name
    return category_dict

categories = load_categories('C:\\Users\\User\\Desktop\\ML Project\\category.txt')
input_folder = 'C:\\Users\\User\\Desktop\\ML Project\\data'
output_folder = os.path.join('C:\\Users\\User\Desktop\\ML Project', 'Cleaned data')

# Ensure the output directory exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Iterate through each category directory
for category_id in range(1, 10):  # Assuming categories are numbered 1 through 9
    category_path = os.path.join(input_folder, str(category_id))
    if os.path.exists(category_path):
        for image_file in os.listdir(category_path):
            if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                image_path = os.path.join(category_path, image_file)
                process_image(image_path, output_folder, category_id, categories)
    else:
        print(f"Category folder {category_path} does not exist.")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, WeightedRandomSampler
from torchvision import transforms, datasets, models
import numpy as np

class ModifiedResNet(nn.Module):
    def __init__(self, num_classes=9):
        super(ModifiedResNet, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        for param in self.resnet.parameters():
            param.requires_grad = False
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

# Define data augmentation and normalization for training
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization
])

# Define the base directory where your dataset is located
base_dir = 'C:\\Users\\User\\Desktop\\ML Project\\Cleaned data'

# Load the dataset with ImageFolder
dataset = datasets.ImageFolder(root=base_dir, transform=transform)

# Calculate class weights
class_counts = torch.tensor([sum(np.array(dataset.targets) == i) for i in range(9)])
class_weights = 1.0 / class_counts.float()
sample_weights = class_weights[dataset.targets]

# Define oversampling and undersampling factors
oversampling_factor = 1.5  # Increase this value for more oversampling
undersampling_factor = 0.5  # Decrease this value for more undersampling

# Apply oversampling and undersampling by adjusting sample weights
for i in range(len(dataset)):
    class_weight = class_weights[dataset.targets[i]]
    sample_weights[i] *= oversampling_factor if class_weight < 1.0 else undersampling_factor

# Create a WeightedRandomSampler
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# Split the dataset into train, validation, and test sets
train_size = int(0.8 * len(dataset))
test_size = int(0.1 * len(dataset))
validation_size = len(dataset) - train_size - test_size
train_dataset, test_dataset, validation_dataset = random_split(dataset, [train_size, test_size, validation_size])

# Setup data loaders with balanced sampling
train_loader = DataLoader(dataset=train_dataset, batch_size=32, sampler=sampler)
validation_loader = DataLoader(dataset=validation_dataset, batch_size=32)
test_loader = DataLoader(dataset=test_dataset, batch_size=32)

# Initialize the network
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ModifiedResNet(num_classes=9).to(device)

# Loss function with class weights
class_weights = class_weights.to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Optimizer (you can choose any optimizer)
optimizer = optim.Adam(model.resnet.fc.parameters(), lr=0.001)

# Print some information (optional)
print(f"Total images in dataset: {len(dataset)}")
print(f"Images in training set: {train_size}")
print(f"Images in validation set: {validation_size}")
print(f"Images in testing set: {test_size}")


In [33]:
import os
import itertools

def evaluate_model(model, loader, device, criterion):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():  # Disable gradient calculation during evaluation
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * images.size(0)  # Accumulate the total loss
            _, predicted = torch.max(outputs.data, 1)
            correct_predictions += (predicted == labels).sum().item()  # Count the number of correct predictions
            total_predictions += labels.size(0)  # Count the total number of predictions

    # Calculate average loss and accuracy
    average_loss = total_loss / len(loader.dataset)
    accuracy = (correct_predictions / total_predictions) * 100

    return average_loss, accuracy


def grid_search_and_save(model, params, train_loader, validation_loader, num_epochs=15, save_path=''):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = torch.nn.CrossEntropyLoss()
    param_combinations = list(itertools.product(*params.values()))
    best_accuracy = 0
    best_params = {}

    for combination in param_combinations:
        param_dict = dict(zip(params.keys(), combination))
        print(f"Testing combination: {param_dict}")
        model.resnet.fc.reset_parameters()
        optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=param_dict['lr'])

        for epoch in range(num_epochs):
            model.train()
            train_total_loss = 0
            train_correct = 0
            train_total = 0
            for images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                train_total_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs.data, 1)
                train_correct += (predicted == labels).sum().item()
                train_total += labels.size(0)
            train_avg_loss = train_total_loss / train_total
            train_accuracy = 100 * train_correct / train_total

            val_loss, val_accuracy = evaluate_model(model, validation_loader, device, criterion)
            print(f"Epoch {epoch+1}/{num_epochs}, LR={param_dict['lr']}, Batch Size={param_dict['batch_size']}: Train Loss {train_avg_loss:.4f}, Train Acc {train_accuracy:.2f}%, Val Loss {val_loss:.4f}, Val Acc {val_accuracy:.2f}%")

            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                best_params = param_dict

    if save_path:
        os.makedirs(save_path, exist_ok=True)
        print(f"Best hyperparameters: {best_params}")
        print(f"Best validation accuracy: {best_accuracy:.2f}%")
        # No model weights are saved in this version

    return best_params, best_accuracy


# Define hyperparameters for grid search
params = {
    'lr': [0.001],
    'batch_size': [32, 64, 128]  # Add batch size options here
}


In [ ]:
# Define hyperparameters for grid search
params = {
    'lr': [0.001,0.01 ,0.1],
    'batch_size': [32, 64, 128]  # Add batch size options here
}

# Initialize the model
model = ModifiedResNet(num_classes=9)

# Define data loaders for training and validation sets
# Note: Make sure to adjust batch size and other parameters as needed
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(dataset=validation_dataset, batch_size=32, shuffle=True)

# Specify the number of epochs for training
num_epochs = 15

# Specify the path where you want to save the best model weights
save_path = 'C:\\Users\\User\\Desktop\\ML Project\\best_model'

# Perform grid search and train the model
best_params, best_accuracy = grid_search_and_save(model, params, train_loader, validation_loader, num_epochs, save_path)

# Print the best hyperparameters and validation accuracy
print("Best Hyperparameters:", best_params)
print("Best Validation Accuracy:", best_accuracy)


In [ ]:
def train_model(model, train_loader, validation_loader, optimizer, criterion, device, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        train_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Calculate average train loss and accuracy for the epoch
        train_loss = train_loss / len(train_loader.dataset)
        train_accuracy = 100 * correct / total

        # Evaluate the model on the validation set
        val_loss, val_accuracy = evaluate_model(model, validation_loader, device, criterion)

        # Print the epoch statistics
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')

    print('Training finished.')

# Define the evaluate_model function as before



# Initialize the network with the best hyperparameters
model = ModifiedResNet(num_classes=9).to(device)

# Load the best model weights
model.load_state_dict(torch.load('C:\\Users\\User\\Desktop\\ML Project\\best_model\\best_model_weights.pth'))

# Define the optimizer with the selected learning rate
optimizer = optim.Adam(model.resnet.fc.parameters(), lr=0.001)

# Train the model using the entire training dataset
# You can adjust the number of epochs and other parameters as needed
train_model(model, train_loader, validation_loader, optimizer, criterion, device, num_epochs=15)

# Evaluate the model on the test set
test_loss, test_accuracy = evaluate_model(model, test_loader, device, criterion)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Function to show image and prediction
def show_prediction(model, loader, device, num_samples=5):
    model.eval()
    classes = loader.dataset.dataset.classes  # Access classes from the original dataset
    with torch.no_grad():
        for i, (images, labels) in enumerate(loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            plt.figure(figsize=(15, 5))  # Increase figure size
            for j in range(min(num_samples, images.size(0))):  # Limit to num_samples or batch size
                plt.subplot(1, num_samples, j+1)
                image = images[j].cpu().numpy().transpose((1, 2, 0))
                plt.imshow(image)
                plt.title(f'Predicted: {classes[predicted[j].item()]}\nActual: {classes[labels[j].item()]}', fontsize=10)  # Adjust fontsize
                plt.axis('off')
            plt.tight_layout()  # Adjust spacing between subplots
            plt.show()
            break  # Show only one batch of images

# Call the function to show predictions
show_prediction(model, validation_loader, device)
